In [ ]:
#!pip3 install langchain_openai

In [ ]:
import os
f = open('openaikey.txt')
os.environ["OPENAI_API_KEY"]=f.read()
os.environ.get("OPENAI_API_KEY")

In [2]:
from langchain_openai import ChatOpenAI
model = ChatOpenAI(temperature=1)

In [3]:
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field,TypeAdapter

class Player(BaseModel):
    
    name: str = Field(description="Name of the player")
    age: str = Field(description="Age of the player")
    specialization: str = Field(description="Sepcialization of the player")
    origin:str = Field(description= "Origin of the player")


class Players(BaseModel):
    players: list = Field(description="Python list of Player")

parser = PydanticOutputParser(pydantic_object=Players)

In [ ]:
parser.get_format_instructions()

In [16]:
from langchain_core.prompts import ChatPromptTemplate

def getChatPrompt(field, country):
    chatpromt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a {field} game Analyst"
        ),
        (
            "human",
            "Get me the name of all the current Playing {country} Players name, their specialization, their age and origin \n {format_instructions}"
        )
    ]
    )

    chatpromtvalue = chatpromt.invoke(
    {
        "field":field,
        "country":country,
        "format_instructions":parser.get_format_instructions()
    }
    ) 
    return chatpromtvalue

In [17]:
chatpromptvalue = getChatPrompt("Carromboard", "India")
chatpromptvalue

ChatPromptValue(messages=[SystemMessage(content='You are a Carromboard game Analyst'), HumanMessage(content='Get me the name of all the current Playing India Players name, their specialization, their age and origin \n The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"players": {"description": "Python list of Player", "items": {}, "title": "Players", "type": "array"}}, "required": ["players"]}\n```')])

In [18]:
result =  model.invoke(chatpromptvalue)
result

AIMessage(content='{\n  "players": [\n    {\n      "name": "Pankaj Advani",\n      "specialization": "Men\'s Singles Champion",\n      "age": 36,\n      "origin": "India"\n    },\n    {\n      "name": "Riya Sisodia",\n      "specialization": "Women\'s Singles Champion",\n      "age": 26,\n      "origin": "India"\n    },\n    {\n      "name": "Arif Akhtar",\n      "specialization": "Men\'s Doubles Champion",\n      "age": 30,\n      "origin": "India"\n    },\n    {\n      "name": "Shruti Sharma",\n      "specialization": "Women\'s Doubles Champion",\n      "age": 28,\n      "origin": "India"\n    }\n  ]\n}', response_metadata={'token_usage': {'completion_tokens': 169, 'prompt_tokens': 196, 'total_tokens': 365}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-e7f67f90-ba43-4d53-ad9e-0d8d7c642ede-0', usage_metadata={'input_tokens': 196, 'output_tokens': 169, 'total_tokens': 365})

In [19]:
players = parser.parse(result.content)
players.players

[{'name': 'Pankaj Advani',
  'specialization': "Men's Singles Champion",
  'age': 36,
  'origin': 'India'},
 {'name': 'Riya Sisodia',
  'specialization': "Women's Singles Champion",
  'age': 26,
  'origin': 'India'},
 {'name': 'Arif Akhtar',
  'specialization': "Men's Doubles Champion",
  'age': 30,
  'origin': 'India'},
 {'name': 'Shruti Sharma',
  'specialization': "Women's Doubles Champion",
  'age': 28,
  'origin': 'India'}]